In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/creditcardfraud


In [6]:
import pandas as pd

data=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
data.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [7]:
# prompt: perform data pre-processsing include comment

# Drop rows with missing values
data.dropna(inplace=True)

# Drop duplicate rows
data.drop_duplicates(inplace=True)

# Separate features (X) and target (y)
X = data.drop('Class', axis=1)
y = data['Class']

# Scale numerical features (optional but recommended for many models)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns) # Convert back to DataFrame

# Display the first 5 rows of the preprocessed data
print("Preprocessed data head:")
print(X_scaled.head())
print("\nTarget variable distribution:")
print(y.value_counts())

Preprocessed data head:
       Time        V1        V2        V3        V4        V5        V6  \
0 -1.996823 -0.701082 -0.041687  1.680101  0.976623 -0.247020  0.348012   
1 -1.996823  0.608792  0.164138  0.109279  0.318998  0.042258 -0.060980   
2 -1.996802 -0.700336 -0.811337  1.174270  0.270648 -0.366756  1.352655   
3 -1.996802 -0.499064 -0.109972  1.187383 -0.608355 -0.008814  0.937245   
4 -1.996781 -0.597606  0.535539  1.025470  0.287092 -0.297036  0.072873   

         V7        V8        V9  ...       V20       V21       V22       V23  \
0  0.193700  0.084434  0.333534  ...  0.326273 -0.024777  0.383483 -0.177444   
1 -0.065656  0.072903 -0.231703  ... -0.089963 -0.311372 -0.881454  0.162081   
2  0.643223  0.210788 -1.381169  ...  0.681564  0.343094  1.065068  1.457772   
3  0.192079  0.320843 -1.264664  ... -0.270428 -0.149093  0.007299 -0.305465   
4  0.481517 -0.228725  0.747917  ...  0.530343 -0.012516  1.101780 -0.220709   

        V24       V25       V26       V27   

In [8]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
data['Amount']=scaler.fit_transform(data[['Amount']])
data['Time']=scaler.fit_transform(data[['Time']])
data["Time"].head(10)

,Time
0,-1.996823
1,-1.996823
2,-1.996802
3,-1.996802
4,-1.996781
5,-1.996781
6,-1.996739
7,-1.996675
8,-1.996675
9,-1.996633


In [9]:
X=data.drop(columns="Class")
y=data["Class"]

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
import numpy as np
def sigmoid(z):
  return 1/(1+np.exp(-z))

def binary_cross_entropy(y_true,y_pred):
    m = len(y_true)
    loss = -(1/m) * np.sum(y_true * np.log(y_pred + 1e-15) + (1 - y_true) * np.log(1 - y_pred + 1e-15))
    return loss

def logistic_regression(X_train,y_train,lr=0.1,epoches=1000):
  m,n=X.shape
  weights=np.zeros(n)
  bias=0
  best_weights=np.zeros(n)
  best_bias=0
  best_loss=float('inf')
  for epoch in range(epoches):
    z=sigmoid(np.dot(X,weights)+bias)
    y_pred=z

    dw=(1/m) * np.dot(X.T, (y_pred - y))
    db=db = (1/m) * np.sum(y_pred - y)

    weights-=lr*dw
    bias-=lr*db
    print(f"Weights at {epoch}: {weights}")
    print(f"Bias at {epoch}: {bias}")
    loss=binary_cross_entropy(y,y_pred)
    if epoch%100==0:
      print(f"Epoch {epoch}, Loss: {loss}")

    if loss<best_loss:
      best_loss=loss
      best_weights=weights
      best_bias=bias
  return best_weights,best_bias


weights, bias = logistic_regression(X_train, y_train, lr=0.1, epoches=1000)
print("Trained best weights:", weights)
print("Trained best bias:", bias)




Streaming output truncated to the last 5000 lines.
Bias at 219: -2.987540250032459
Weights at 220: [-2.42285432e-03 -1.69940297e-02  1.96381558e-02 -4.20563868e-02
  3.68550869e-02 -1.71414294e-02 -1.06662635e-02 -4.38348567e-02
  1.75644229e-03 -2.57377068e-02 -5.72550915e-02  4.34297406e-02
 -7.24540095e-02 -3.30908080e-03 -9.21650882e-02 -2.89194598e-03
 -5.49434613e-02 -8.62749718e-02 -2.99715211e-02  9.16115906e-03
  4.71387497e-03  9.85917280e-03  2.14298743e-03 -1.63331394e-03
 -2.84607310e-03  4.36718076e-04  9.38061508e-04 -1.51331087e-03
  8.24625752e-05  2.92417281e-03]
Bias at 220: -2.9922344600509185
Weights at 221: [-2.42444471e-03 -1.70058884e-02  1.96628191e-02 -4.21174291e-02
  3.69372455e-02 -1.71411590e-02 -1.06881964e-02 -4.38931565e-02
  1.72838593e-03 -2.57954690e-02 -5.73729936e-02  4.35259718e-02
 -7.26101047e-02 -3.32119330e-03 -9.23931469e-02 -2.89626695e-03
 -5.50505072e-02 -8.64335683e-02 -3.00172343e-02  9.17498035e-03
  4.72098590e-03  9.88380360e-03  2.15

In [1]:
def predict(X, weights, bias):
    z = np.dot(X, weights) + bias
    y_pred = sigmoid(z)
    return [1 if p > 0.5 else 0 for p in y_pred]
y_pred = predict(X_test, best_weights, best_bias)


NameError: name 'X_test' is not defined